In [ ]:
import cv2
import numpy as np
import os
import tkinter as tk
from tkinter import filedialog, messagebox

# Biến đếm để kiểm tra số lượng mẫu đã thu thập
sample_count = 0

# Kích thước khung hình chữ nhật
RECT_WIDTH = 300
RECT_HEIGHT = 350

# Bước 1: Thu thập mẫu khuôn mặt
def capture_samples(person_name, sample_count_required=200):
    global sample_count
    sample_dir = f'samples/{person_name}'
    os.makedirs(sample_dir, exist_ok=True)

    cap = cv2.VideoCapture(0)

    while sample_count < sample_count_required:
        ret, frame = cap.read()
        if not ret:
            break

        # Lật lại hình ảnh
        frame = cv2.flip(frame, 1)

        cv2.imshow('Capture Samples', frame)

        # Vẽ khung hình chữ nhật ở giữa màn hình, vùng trong khung hình chữ nhật sẽ được lưu làm mẫu thay vì toàn bộ khung hình.
        height, width, _ = frame.shape
        top_left = ((width - RECT_WIDTH) // 2, (height - RECT_HEIGHT) // 2)
        bottom_right = (top_left[0] + RECT_WIDTH, top_left[1] + RECT_HEIGHT)
        cv2.rectangle(frame, top_left, bottom_right, (0, 255, 0), 2)

        cv2.imshow('Capture Samples', frame)

        key = cv2.waitKey(1)
        if key == ord('c'):
            if sample_count < sample_count_required:
                # Lưu chỉ vùng trong khung hình chữ nhật
                roi = frame[top_left[1]:bottom_right[1], top_left[0]:bottom_right[0]]
                cv2.imwrite(f'{sample_dir}/sample_{sample_count + 1}.jpg', roi)
                print(f'Saved {sample_dir}/sample_{sample_count + 1}.jpg')
                sample_count += 1

                # Hiển thị số lượng mẫu đã chụp
                cv2.putText(frame, f'Samples: {sample_count}/{sample_count_required}', (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                cv2.imshow('Capture Samples', frame)
                cv2.waitKey(500)  # Chờ 0.5 giây trước khi chụp tiếp để người dùng chuẩn bị

        if key == ord('q') or sample_count >= sample_count_required:
            break

    cap.release()
    cv2.destroyAllWindows()

# Bước 2: Đào tạo bộ nhận diện khuôn mặt
def train_recognizer():
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    recognizer = cv2.face.LBPHFaceRecognizer_create()

    samples_dir = 'samples'
    face_samples = []
    ids = []
    names = []

    for person_name in os.listdir(samples_dir):
        person_dir = os.path.join(samples_dir, person_name)
        if not os.path.isdir(person_dir):
            continue

        person_id = len(names)
        names.append(person_name)

        for filename in os.listdir(person_dir):
            if filename.endswith('.jpg'):
                img_path = os.path.join(person_dir, filename)
                img = cv2.imread(img_path)
                gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

                faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

                for (x, y, w, h) in faces:
                    face_samples.append(gray[y:y+h, x:x+w])
                    ids.append(person_id)

    recognizer.train(face_samples, np.array(ids))
    recognizer.save('face_recognizer.yml')
    return names

# Bước 3: Cập nhật code để nhận diện khuôn mặt
def recognize_faces_in_video(video_path, names):
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    recognizer = cv2.face.LBPHFaceRecognizer_create()
    recognizer.read('face_recognizer.yml')

    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    def on_trackbar(val):
        cap.set(cv2.CAP_PROP_POS_FRAMES, val)

    cv2.namedWindow('Face Recognition and Tracking', cv2.WINDOW_NORMAL)
    cv2.createTrackbar('Position', 'Face Recognition and Tracking', 0, total_frames, on_trackbar)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Lật lại hình ảnh
        frame = cv2.flip(frame, 1)

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))


        for (x, y, w, h) in faces:
            face_id, confidence = recognizer.predict(gray[y:y+h, x:x+w])
            if confidence < 50:  # Lower confidence means higher certainty
                name = names[face_id]
                cv2.putText(frame, f'Name: {name}', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)
            else:
                cv2.putText(frame, f'Name: Unidentified', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)

        cv2.imshow('Face Recognition and Tracking', frame)

        current_frame = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
        cv2.setTrackbarPos('Position', 'Face Recognition and Tracking', current_frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Định nghĩa các hàm GUI và giao diện người dùng
def capture_samples_gui():
    person_name = entry_name.get()
    if not person_name:
        messagebox.showwarning("Warning", "Please enter a name!")
        return
    global sample_count
    sample_count = 0  # Reset sample count
    capture_samples(person_name)

def train_recognizer_gui():
    names = train_recognizer()
    messagebox.showinfo("Info", "Training complete!")

def recognize_faces_in_video_gui():
    video_path = filedialog.askopenfilename()
    if not video_path:
        return
    names = train_recognizer()
    recognize_faces_in_video(video_path, names)

# Tạo giao diện người dùng với tkinter
root = tk.Tk()
root.title("Face Recognition System")

# Lấy kích thước màn hình
screen_width = root.winfo_screenwidth()
screen_height = root.winfo_screenheight()

# Kích thước của frame
frame_width = 400  # Thay đổi chiều rộng nếu cần
frame_height = 400  # Thay đổi chiều cao nếu cần

# Tính toán vị trí để canh giữa màn hình
x = (screen_width - frame_width) // 2
y = (screen_height - frame_height) // 2

# Đặt vị trí của cửa sổ root
root.geometry(f"{frame_width}x{frame_height}+{x}+{y}")

# Frame chính với màu nền
frame = tk.Frame(root, bg='#e6e6e6', padx=20, pady=20)
frame.pack(fill=tk.BOTH, expand=True)

# Tiêu đề
label_title = tk.Label(frame, text="Face Recognition System", font=("Helvetica", 18), pady=20)
label_title.pack()

# Nhập tên người dùng
label_name = tk.Label(frame, text="Enter Name:", font=("Helvetica", 12))
label_name.pack()

entry_name = tk.Entry(frame, font=("Helvetica", 12), width=30)
entry_name.pack()

# Các nút chức năng
btn_capture_samples = tk.Button(frame, text="Capture Samples", font=("Helvetica", 12), command=capture_samples_gui)
btn_capture_samples.pack(pady=10)

btn_train_recognizer = tk.Button(frame, text="Train Recognizer", font=("Helvetica", 12), command=train_recognizer_gui)
btn_train_recognizer.pack(pady=10)

btn_recognize_faces = tk.Button(frame, text="Recognize Faces in Video", font=("Helvetica", 12), command=recognize_faces_in_video_gui)
btn_recognize_faces.pack(pady=10)

# Chạy giao diện người dùng
root.mainloop()